# Midterm Expectation

- Baseline
- Experiment (i.e., something different; e.x., spreadsheet)
- Timeline (what we have done, and what we will be doing)

# Dependencies

In [ ]:
import os

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import (
    CenterCrop,
    Compose,
    RandAugment,
    RandomHorizontalFlip,
    RandomRotation,
    ToTensor,
)
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import math

# Mount Google Drive

## Install gdfuse

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.

## Fetch API Key

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

/content
mkdir: cannot create directory ‘drive’: File exists
/content/drive
mkdir: cannot create directory ‘MyDrive’: File exists
/content
/
Access token retrieved correctly.


# Unzip Data

In [ ]:
! pip install gdown
! cd /content/ && gdown https://drive.google.com/uc?id=1zD7fUAt12L16ywPSjRsj4IqL-Lqg13zK
# ! cp /content/drive/MyDrive/11785/project/data.zip /content/
! cd /content && unzip data.zip
! rm /content/data.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/imagery/realsense_overhead/train/dish_1562169840/rgb.png  
   creating: data/imagery/realsense_overhead/train/dish_1562170163/
  inflating: data/imagery/realsense_overhead/train/dish_1562170163/depth_color.png  
  inflating: data/imagery/realsense_overhead/train/dish_1562170163/depth_raw.png  
  inflating: data/imagery/realsense_overhead/train/dish_1562170163/rgb.png  
   creating: data/imagery/realsense_overhead/train/dish_1562170236/
  inflating: data/imagery/realsense_overhead/train/dish_1562170236/depth_color.png  
  inflating: data/imagery/realsense_overhead/train/dish_1562170236/depth_raw.png  
  inflating: data/imagery/realsense_overhead/train/dish_1562170236/rgb.png  
   creating: data/imagery/realsense_overhead/train/dish_1562170331/
  inflating: data/imagery/realsense_overhead/train/dish_1562170331/depth_color.png  
  inflating: data/imagery/realsense_overhead/train/dish_1562170331/depth_raw.png  
  inflatin

# Global Variables

In [ ]:
DATA_PATH = "/content/data"
METADATA_DIR = f"{DATA_PATH}/metadata"
IMAGERY_DIR = f"{DATA_PATH}/imagery/realsense_overhead"

# Helper Functions for Data

In [ ]:
def read_csv_variable_cols(filepath: str) -> pd.DataFrame:
    """https://stackoverflow.com/a/57824142.
    We only read the first 6 columns to retrieve required labels.
    """
    ### Loop the data lines
    with open(filepath, 'r') as temp_f:
        # get No of columns in each line
        col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

    ### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
    column_names = [i for i in range(0, max(col_count))]

    ### Read csv
    return pd.read_csv(filepath, header=None, delimiter=",", names=column_names, low_memory=False).iloc[:,:6]

# Ingredient and Dish Metadata (Groun Truths)

## Data Format

### Training-testing data

- "imagery/realsense_overhead/dish_<id>" contains the images as the input data
- "dish_ids" contains training-testing splits

### Labels (metadata)

All labels need to be preprocessed. For each dish, we need to extract the following:
- total calorie
- mass (optional according to the paper)
- the amount of the three macronutrients (fat, carb, protein)

It doesn't seem that bad - we don't need to process the ingredients because they are purely there for constructing the labels. For our multi-task learning, we only need to have the above labels. The three tasks are:

1. Calorie
2. Macronutrients (fat, carb, protein)
3. Mass (optional)

这样一来我们可以把labels和image data放在一起，每次返回input和expected output.

## Dish Metadata

In [ ]:
# Metadata for dishes has variable numbers of columns per row.
# Can do similar stuff to dish_metadata_cafe2.csv
# The first 6 columns: [dish_id, total_calories, total_mass, total_fat, total_carb, total_protein]
dish_metadata_1 = read_csv_variable_cols(f"{METADATA_DIR}/dish_metadata_cafe1.csv")
# Rename the columns
dish_metadata_1 = dish_metadata_1.rename(columns={0:"dish_id", 1:"total_calories", 2:"total_mass", 3:"total_fat", 4:"total_carb", 5:"total_protein"})

dish_metadata_2 = read_csv_variable_cols(f"{METADATA_DIR}/dish_metadata_cafe2.csv")
# Rename the columns
dish_metadata_2 = dish_metadata_2.rename(columns={0:"dish_id", 1:"total_calories", 2:"total_mass", 3:"total_fat", 4:"total_carb", 5:"total_protein"})

dish_metadata = pd.concat((dish_metadata_1, dish_metadata_2), ignore_index=True)
# Convert to dictionary
labels_dict = dish_metadata.set_index("dish_id").to_dict("index")

# Hyperparameters

In [ ]:
config = {
    'epochs': 150,
    'batch_size': 32,
    'lr': 2e-4,
}

class Config:
    def __init__(self, config):
        for k, v in config.items():
            setattr(self, k, v)

config = Config(config)

transforms = Compose([CenterCrop((256, 256)), RandAugment(3), ToTensor()])

# Datasets and DataLoaders

In [ ]:
class RGBDataset(Dataset):
    """4.2 The input resolution to the
    network is a 256x256 image, where images were downsized
    and center cropped in order to retain the most salient dish
    region.

    我们baseline应该只用RGB就行 (根据4.2).
    """

    def __init__(self, data_dir, transforms=transforms, labels=labels_dict, train=True):
        self.data_dir = data_dir
        self.transforms = transforms
        self.labels = labels
        self.train = train

        # # ['dish_1556572657', 'dish_1556573514', 'dish_1556575014', 'dish_1556575083', 'dish_1556575124', 'dish_1556575273', 'dish_1556575327']
        dirs = os.listdir(self.data_dir)

        self.dish_ids = []
        for dir in dirs:
            if "rgb.png" in os.listdir(os.path.join(self.data_dir,dir)):
                self.dish_ids.append(dir)

        self.dish_ids.sort()

        self.img_paths = list(
            map(
                lambda fname: os.path.join(self.data_dir, fname),
                self.dish_ids,
            )
        )

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        rgb_path = f"{self.img_paths[idx]}/rgb.png"
        dish_id = self.dish_ids[idx]
        transformed_img = self.transforms(Image.open(rgb_path))
        if self.train:
            label = torch.tensor(list(self.labels[dish_id].values()))
            return transformed_img, label
        else:
            return transformed_img

In [ ]:
TRAIN_DIR = f"{IMAGERY_DIR}/train"
VALID_DIR = f"{IMAGERY_DIR}/test"
# TEST_DIR = IMAGERY_DIR

train_dataset = RGBDataset(TRAIN_DIR, labels=labels_dict)
valid_dataset = RGBDataset(VALID_DIR, labels=labels_dict)
# test_dataset = RGBDataset(TEST_DIR, train=False)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)
# test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

num_training_batches = len(train_loader)

# InceptionV2

In [ ]:
def ConvBNReLU(in_channels,out_channels,kernel_size,stride=1,padding=0):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU6(inplace=True),
    )

def ConvBNReLUFactorization(in_channels,out_channels,kernel_sizes,paddings):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_sizes, stride=1,padding=paddings),
        nn.BatchNorm2d(out_channels),
        nn.ReLU6(inplace=True)
    )

class InceptionV2ModuleA(nn.Module):
    def __init__(self, in_channels,out_channels1,out_channels2reduce, out_channels2, out_channels3reduce, out_channels3, out_channels4):
        super(InceptionV2ModuleA, self).__init__()

        self.branch1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels1,kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1),
            ConvBNReLU(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_size=3, padding=1),
        )

        self.branch3 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels,out_channels=out_channels3reduce,kernel_size=1),
            ConvBNReLU(in_channels=out_channels3reduce, out_channels=out_channels3, kernel_size=3, padding=1),
            ConvBNReLU(in_channels=out_channels3, out_channels=out_channels3, kernel_size=3, padding=1),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels4, kernel_size=1),
        )

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out4 = self.branch4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

class InceptionV2ModuleB(nn.Module):
    def __init__(self, in_channels,out_channels1,out_channels2reduce, out_channels2, out_channels3reduce, out_channels3, out_channels4):
        super(InceptionV2ModuleB, self).__init__()

        self.branch1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels1,kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1),
            ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2reduce, kernel_sizes=[1,3],paddings=[0,1]),
            ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_sizes=[3,1],paddings=[1, 0]),
        )

        self.branch3 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels,out_channels=out_channels3reduce,kernel_size=1),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3reduce,kernel_sizes=[1, 3], paddings=[0, 1]),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3reduce,kernel_sizes=[3, 1], paddings=[1, 0]),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3reduce, kernel_sizes=[1, 3], paddings=[0, 1]),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3,kernel_sizes=[3, 1], paddings=[1, 0]),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels4, kernel_size=1),
        )

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out4 = self.branch4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

class InceptionV2ModuleC(nn.Module):
    def __init__(self, in_channels,out_channels1,out_channels2reduce, out_channels2, out_channels3reduce, out_channels3, out_channels4):
        super(InceptionV2ModuleC, self).__init__()

        self.branch1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels1,kernel_size=1)

        self.branch2_conv1 = ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1)
        self.branch2_conv2a = ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_sizes=[1,3],paddings=[0,1])
        self.branch2_conv2b = ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_sizes=[3,1],paddings=[1,0])

        self.branch3_conv1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels3reduce,kernel_size=1)
        self.branch3_conv2 = ConvBNReLU(in_channels=out_channels3reduce, out_channels=out_channels3, kernel_size=3,stride=1,padding=1)
        self.branch3_conv3a = ConvBNReLUFactorization(in_channels=out_channels3, out_channels=out_channels3, kernel_sizes=[3, 1],paddings=[1, 0])
        self.branch3_conv3b = ConvBNReLUFactorization(in_channels=out_channels3, out_channels=out_channels3, kernel_sizes=[1, 3],paddings=[0, 1])

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels4, kernel_size=1),
        )

    def forward(self, x):
        out1 = self.branch1(x)
        x2 = self.branch2_conv1(x)
        out2 = torch.cat([self.branch2_conv2a(x2), self.branch2_conv2b(x2)],dim=1)
        x3 = self.branch3_conv2(self.branch3_conv1(x))
        out3 = torch.cat([self.branch3_conv3a(x3), self.branch3_conv3b(x3)], dim=1)
        out4 = self.branch4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

class InceptionV3ModuleD(nn.Module):
    def __init__(self, in_channels,out_channels1reduce,out_channels1,out_channels2reduce, out_channels2):
        super(InceptionV3ModuleD, self).__init__()

        self.branch1 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels1reduce, kernel_size=1),
            ConvBNReLU(in_channels=out_channels1reduce, out_channels=out_channels1, kernel_size=3,stride=2,padding=1)
        )

        self.branch2 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1),
            ConvBNReLU(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=out_channels2, out_channels=out_channels2, kernel_size=3, stride=2,padding=1),
        )

        self.branch3 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out = torch.cat([out1, out2, out3], dim=1)
        return out

class InceptionAux(nn.Module):
    def __init__(self, in_channels,out_channels):
        super(InceptionAux, self).__init__()

        self.auxiliary_avgpool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.auxiliary_conv1 = ConvBNReLU(in_channels=in_channels, out_channels=128, kernel_size=1)
        self.auxiliary_conv2 = nn.Conv2d(in_channels=128, out_channels=768, kernel_size=5,stride=1)
        self.auxiliary_dropout = nn.Dropout(p=0.7)
        self.auxiliary_linear1 = nn.Linear(in_features=768, out_features=out_channels)

    def forward(self, x):
        x = self.auxiliary_conv1(self.auxiliary_avgpool(x))
        x = self.auxiliary_conv2(x)
        x = x.view(x.size(0), -1)
        out = self.auxiliary_linear1(self.auxiliary_dropout(x))
        return out

class InceptionV2(nn.Module):
    def __init__(self, num_classes=1000, stage='train'):
        super(InceptionV2, self).__init__()
        self.stage = stage

        self.block1 = nn.Sequential(
            ConvBNReLU(in_channels=3, out_channels=64, kernel_size=7,stride=2,padding=3),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        )

        self.block2 = nn.Sequential(
            ConvBNReLU(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2,padding=1),
        )

        self.block3 = nn.Sequential(
            InceptionV2ModuleA(in_channels=192,out_channels1=64,out_channels2reduce=64, out_channels2=64, out_channels3reduce=64, out_channels3=96, out_channels4=32),
            InceptionV2ModuleA(in_channels=256, out_channels1=64, out_channels2reduce=64, out_channels2=96,out_channels3reduce=64, out_channels3=96, out_channels4=64),
            InceptionV3ModuleD(in_channels=320, out_channels1reduce=128, out_channels1=160, out_channels2reduce=64,out_channels2=96),
        )

        self.block4 = nn.Sequential(
            InceptionV2ModuleB(in_channels=576, out_channels1=224, out_channels2reduce=64, out_channels2=96,out_channels3reduce=96, out_channels3=128, out_channels4=128),
            InceptionV2ModuleB(in_channels=576, out_channels1=192, out_channels2reduce=96, out_channels2=128,out_channels3reduce=96, out_channels3=128, out_channels4=128),
            InceptionV2ModuleB(in_channels=576, out_channels1=160, out_channels2reduce=128, out_channels2=160,out_channels3reduce=128, out_channels3=128, out_channels4=128),
            InceptionV2ModuleB(in_channels=576, out_channels1=96, out_channels2reduce=128, out_channels2=192,out_channels3reduce=160, out_channels3=160, out_channels4=128),
            InceptionV3ModuleD(in_channels=576, out_channels1reduce=128, out_channels1=192, out_channels2reduce=192,out_channels2=256),
        )

        self.block5 = nn.Sequential(
            InceptionV2ModuleC(in_channels=1024, out_channels1=352, out_channels2reduce=192, out_channels2=160,out_channels3reduce=160, out_channels3=112, out_channels4=128),
            InceptionV2ModuleC(in_channels=1024, out_channels1=352, out_channels2reduce=192, out_channels2=160,
                               out_channels3reduce=192, out_channels3=112, out_channels4=128)
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((2,2))
        self.dropout = nn.Dropout(p=0.5)
    

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avg_pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        return x

# Resnet50

In [ ]:
import torch.utils.model_zoo as model_zoo
model_urls = {
    'resnet18': 'https://s3.amazonaws.com/pytorch/models/resnet18-5c106cde.pth',
    'resnet34': 'https://s3.amazonaws.com/pytorch/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://s3.amazonaws.com/pytorch/models/resnet50-19c8e357.pth',
    'resnet101': 'https://s3.amazonaws.com/pytorch/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://s3.amazonaws.com/pytorch/models/resnet152-b121ed2d.pth',
}

class Bottleneck(nn.Module):
  expansion = 4

  def __init__(self, inplanes, planes, stride=1, downsample=None):
    super(Bottleneck, self).__init__()
    
    self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False) # change
    self.bn1 = nn.BatchNorm2d(planes)
    
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, # change
                 padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)
    
    self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm2d(planes * 4)
    self.relu = nn.ReLU(inplace=True)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x):
    residual = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)

    out = self.conv3(out)
    out = self.bn3(out)

    if self.downsample is not None:
      residual = self.downsample(x)

    out += residual
    out = self.relu(out)

    return out
    
    
class ResNet(nn.Module):
  def __init__(self, block, layers, num_classes=1000):
    self.inplanes = 64
    super().__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                 bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=True) # change
    self.layer1 = self._make_layer(block, 64, layers[0])
    self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
    self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
    self.layer4 = self._make_layer(block, 512, layers[3], stride=2)   # different
    self.avgpool = nn.AvgPool2d(7)
    self.fc = nn.Linear(512 * block.expansion, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()

  def _make_layer(self, block, planes, blocks, stride=1):
    downsample = None
    if stride != 1 or self.inplanes != planes * block.expansion:
      downsample = nn.Sequential(
        nn.Conv2d(self.inplanes, planes * block.expansion,
              kernel_size=1, stride=stride, bias=False),
        nn.BatchNorm2d(planes * block.expansion),
      )

    layers = []
    layers.append(block(self.inplanes, planes, stride, downsample))
    self.inplanes = planes * block.expansion
    for i in range(1, blocks):
      layers.append(block(self.inplanes, planes))

    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)

    return x

def resnet50(pretrained=True):
  """Constructs a ResNet-50 model.
  Args:
    pretrained (bool): If True, returns a model pre-trained on ImageNet
  """
  model = ResNet(Bottleneck, [3, 4, 6, 3])
  if pretrained:
    model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
  return model


# Baseline Model

In [ ]:
class BaseNet(nn.Module):
    def __init__(self):
        super().__init__()
        # self.backbone = InceptionV2([1, 1, 3])
        self.backbone = resnet50()
        self.fc = nn.Linear(1000, 4096) 
        self.fc1 = nn.Linear(4096, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc_calories = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.Linear(4096, 1)
        )
        self.fc_mass = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.Linear(4096, 1)
        )
        self.fc_mc = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.Linear(4096, 3)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x) # Comment it if backbone is resnet
        x = self.fc2(self.fc1(x))

        x_cal = self.fc_calories(x)
        x_mass = self.fc_mass(x)
        x_mn = self.fc_mc(x)

        return x_cal, x_mass, x_mn

# Multi-task Loss

In [ ]:
class MultiTaskLearner(nn.Module):
    def __init__(self, model: nn.Module):
        super(MultiTaskLearner, self).__init__()
        self.model = model
        self.criterion = nn.L1Loss()

    def forward(self, x, y):
        # 1 x 5 Tensor [total_calories, total_mass, total_fat, total_carb, total_protein]

        out_cal, out_mass, out_mn = self.model(x)

        loss_calorie = self.criterion(out_cal, y[:, 0:1])
        
        loss_mass = self.criterion(out_mass, y[:, 1:2])

        loss_mn = self.criterion(out_mn, y[:, 2:])

        loss_total = loss_calorie + loss_mass + loss_mn

        return loss_total

# Delete Model

In [ ]:
# del model

# Utility Funs

In [ ]:
CHECKPOINT_PATH = "/content/drive/MyDrive/checkpoints"
def create_dir_if_not_exists(dirpath: str):
    """Create the specified directory with all intermediate directories if necessary."""
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

def save_checkpoint(
    epoch: int,
    loss: float,
    model: nn.Module,
    model_name: str,
    checkpoint_path: str = CHECKPOINT_PATH,
):
    create_dir_if_not_exists(checkpoint_path)
    torch.save(
        {"epoch": epoch, "loss": loss, "model_state_dict": model.state_dict()},
        os.path.join(checkpoint_path, model_name),
    )


def load_checkpoint(filepath: str):
    state_dict = torch.load(filepath)
    epoch, loss, model_state_dict = (
        state_dict["epoch"],
        state_dict["loss"],
        state_dict["model_state_dict"],
    )
    return epoch, loss, model_state_dict

# Training

In [ ]:
print(f"Cuda is available: {torch.cuda.is_available()}")
model = BaseNet()
model.cuda()
learner = MultiTaskLearner(model)

optimizer = torch.optim.Adam(model.parameters(), config.lr, weight_decay=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.epochs*len(train_loader))
scaler = torch.cuda.amp.GradScaler()

Cuda is available: True


Downloading: "https://s3.amazonaws.com/pytorch/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
for epoch in range(config.epochs):
    model.train()
    total_loss = 0

    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.cuda()
        y = y.cuda()

        # with torch.cuda.amp.autocast():     
        loss = learner(x, y)

        # Update # correct & loss as we go
        total_loss += float(loss)

        # Compute training metrics
        train_loss = float(total_loss / (i + 1))
        cur_lr = float(optimizer.param_groups[0]['lr'])


        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(train_loss),
            lr="{:.04f}".format(cur_lr))
        
        # Another couple things you need for FP16. 
        # scaler.scale(loss).backward() # This is a replacement for loss.backward()
        loss.backward()
        # scaler.step(optimizer) # This is a replacement for optimizer.step()
        optimizer.step()
        # scaler.update() # This is something added just for FP16

        scheduler.step() # We told scheduler T_max that we'd call step() (len(train_loader) * epochs) many times.

        batch_bar.update() # Update tqdm bar

    batch_bar.close() # You need this to close the tqdm bar

    train_loss = total_loss / len(train_loader)
    
    # Save the model every 3 epochs
    if epoch % 3 == 0:
        save_checkpoint(epoch, train_loss, model, "resnet")

    # You can add validation per-epoch here if you would like
    model.eval()
    batch_bar = tqdm(total=len(valid_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
    total_loss = 0
    for i, (x, y) in enumerate(valid_loader):

        x = x.cuda()
        y = y.cuda()

        with torch.no_grad():
            loss = learner(x, y)
        

        total_loss += float(loss)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))))

        batch_bar.update()
        
    batch_bar.close()

    # scheduler.step(float(total_loss / (i + 1)))

    valid_loss = total_loss / len(valid_loader)

    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}, Valid Loss {:.04f}".format(
        epoch + 1, config.epochs, train_loss, cur_lr, valid_loss))


Epoch 1/150: Train Loss 230.9911, Learning Rate 0.0002, Valid Loss 267.3615


Epoch 2/150: Train Loss 197.0106, Learning Rate 0.0002, Valid Loss 232.3654


Epoch 3/150: Train Loss 183.9895, Learning Rate 0.0002, Valid Loss 302.6214


Epoch 4/150: Train Loss 183.9020, Learning Rate 0.0002, Valid Loss 220.8743


Epoch 5/150: Train Loss 180.3823, Learning Rate 0.0002, Valid Loss 254.8966


...

Val:   0%|          | 0/29 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f15795cf560>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f15795cf560>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers


Epoch 148/150: Train Loss 110.8244, Learning Rate 0.0000, Valid Loss 223.9793


Train:   0%|          | 0/81 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f15795cf560>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f15795cf560>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_worker

Epoch 149/150: Train Loss 110.8650, Learning Rate 0.0000, Valid Loss 221.2720


Epoch 150/150: Train Loss 110.3283, Learning Rate 0.0000, Valid Loss 225.4630


# Evaluate Stats

## Copy Ground Truths to Colab 

In [ ]:
! cp /content/drive/MyDrive/11785/project/ground_truth.csv /content/data/metadata/

## Dataset for Evaluation

In [ ]:
class EvalDataset(Dataset):

    def __init__(self, data_dir, transforms=Compose([CenterCrop((256, 256)), ToTensor()]), labels=labels_dict):
        self.data_dir = data_dir
        self.transforms = transforms
        self.labels = labels

        # # ['dish_1556572657', 'dish_1556573514', 'dish_1556575014', 'dish_1556575083', 'dish_1556575124', 'dish_1556575273', 'dish_1556575327']
        dirs = os.listdir(self.data_dir)

        self.dish_ids = []
        for dir in dirs:
            if "rgb.png" in os.listdir(os.path.join(self.data_dir,dir)):
                self.dish_ids.append(dir)

        self.dish_ids.sort()

        self.img_paths = list(
            map(
                lambda fname: os.path.join(self.data_dir, fname),
                self.dish_ids,
            )
        )

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        rgb_path = f"{self.img_paths[idx]}/rgb.png"
        dish_id = self.dish_ids[idx]
        transformed_img = self.transforms(Image.open(rgb_path))
        # Return the dish id for writing the csv file
        return transformed_img, dish_id

In [ ]:
torch.cuda.empty_cache()
eval_train_data = EvalDataset(TRAIN_DIR, labels=labels_dict)
eval_val_data = EvalDataset(VALID_DIR, labels=labels_dict)

eval_train_loader = DataLoader(eval_train_data, batch_size=config.batch_size, shuffle=False, num_workers=2)
eval_val_loader = DataLoader(eval_val_data, batch_size=config.batch_size, shuffle=False, num_workers=2)

## (Optional) Load Checkpoint

In [ ]:
model = BaseNet()
_, _, model_state_dict = load_checkpoint(f"{CHECKPOINT_PATH}/baseline")
model.load_state_dict(model_state_dict)
model.cuda()

BaseNet(
  (backbone): InceptionV2(
    (block1): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (block2): Sequential(
      (0): Sequential(
        (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (block3): Sequential(
      (0): InceptionV2ModuleA(
        (branch1): Sequential(
          (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

## Inference on training and testing data

In [ ]:
prediction_filepath = "/content/data/metadata/outputs.csv"
# if os.path.exists(prediction_filepath):
#     os.remove(prediction_filepath)

model.eval()
batch_bar = tqdm(total=len(eval_train_loader), dynamic_ncols=True, position=0, leave=False, desc='Eval_Train')
results_all = None
# Inference on training data
for i, (x, y) in enumerate(eval_train_loader):

    x = x.cuda()
    dish_ids = np.array(list(y))
    dish_ids = dish_ids.reshape(dish_ids.shape[0],1)

    cal, mass, mn = model(x)

    results = torch.cat((cal,mass,mn), 1).detach().cpu().numpy()

    results = np.concatenate((dish_ids, results), 1)
    
    if results_all is None:
        results_all = results
    else:
        results_all = np.concatenate((results_all, results), 0)

    del cal, mass, mn
    torch.cuda.empty_cache()

    batch_bar.update()
    
batch_bar.close()
# Inference on validation data
batch_bar = tqdm(total=len(eval_val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
for i, (x, y) in enumerate(eval_val_loader):

    x = x.cuda()
    dish_ids = np.array(list(y))
    dish_ids = dish_ids.reshape(dish_ids.shape[0],1)

    cal, mass, mn = model(x)

    results = torch.cat((cal,mass,mn), 1).detach().cpu().numpy()

    results = np.concatenate((dish_ids, results), 1)
    
    if results_all is None:
        results_all = results
    else:
        results_all = np.concatenate((results_all, results), 0)

    del cal, mass, mn
    torch.cuda.empty_cache()

    batch_bar.update()
    
batch_bar.close()

In [ ]:
# Write to csv
np.savetxt(prediction_filepath, results_all, delimiter=",", fmt='%s,%s,%s,%s,%s,%s')

In [ ]:
r"""Script to compute statistics on nutrition predictions.

This script takes in a csv of nutrition predictions and computes absolute and
percentage mean average error values comparable to the metrics used to eval
models in the Nutrition5k paper. The input csv file of nutrition predictions
should be in the form of:
dish_id, calories, mass, carbs, protein
And the groundtruth values will be pulled from the metadata csv file provided
in the Nutrition5k dataset release where the first 5 fields are also:
dish_id, calories, mass, carbs, protein

Example Usage:
python compute_statistics.py path/to/groundtruth.csv path/to/predictions.csv \
path/to/output_statistics.json
"""

import json
from os import path
import statistics
import sys

DISH_ID_INDEX = 0
DATA_FIELDNAMES = ["dish_id", "calories", "mass", "fat", "carb", "protein"]


def ReadCsvData(filepath):
  if not path.exists(filepath):
    raise Exception("File %s not found" % path)
  parsed_data = {}
  with open(filepath, "r") as f_in:
    filelines = f_in.readlines()
    for line in filelines:
      data_values = line.strip().split(",")
      parsed_data[data_values[DISH_ID_INDEX]] = data_values
  return parsed_data

groundtruth_csv_path = "/content/data/metadata/ground_truth.csv"
predictions_csv_path = prediction_filepath
output_path = "/content/data/metadata/eval_results.json"

groundtruth_data = ReadCsvData(groundtruth_csv_path)
prediction_data = ReadCsvData(predictions_csv_path)

groundtruth_values = {}
err_values = {}
output_stats = {}

for field in DATA_FIELDNAMES[1:]:
  groundtruth_values[field] = []
  err_values[field] = []

for dish_id in prediction_data:
  for i in range(1, len(DATA_FIELDNAMES)):
    groundtruth_values[DATA_FIELDNAMES[i]].append(
        float(groundtruth_data[dish_id][i]))
    err_values[DATA_FIELDNAMES[i]].append(abs(
        float(prediction_data[dish_id][i])
        - float(groundtruth_data[dish_id][i])))

for field in DATA_FIELDNAMES[1:]:
  output_stats[field + "_MAE"] = statistics.mean(err_values[field])
  output_stats[field + "_MAE_%"] = (100 * statistics.mean(err_values[field]) /
                                    statistics.mean(groundtruth_values[field]))

with open(output_path, "w") as f_out:
  f_out.write(json.dumps(output_stats))
